In [1]:
import numpy as np
import pandas as pd
import datetime

## 기존 데이터프레임

In [2]:
batter = pd.read_csv('private_batter.csv')
batter = batter.sort_values(by=['P_ID','GDAY_DS'])

In [3]:
batter['GDAY_DS'] = pd.to_datetime(batter['GDAY_DS'],format='%Y-%m-%d')

In [4]:
batter[batter['P_ID']==50054].head()

,GDAY_DS,T_ID,P_ID,TB_SC,PA-AB,AB,RUN,RBI,HIT,SH+SF,KK,AVG,SB_trial,BABIP
76138,2020-06-03,KT,50054,B,0,3,0,0,2,0,0,0.666667,0,0.666667
76262,2020-06-04,KT,50054,B,1,3,1,0,0,0,1,0.000000,0,0.000000
76522,2020-06-06,KT,50054,T,0,0,0,0,0,0,0,0.000000,0,0.000000
76647,2020-06-07,KT,50054,T,0,3,0,0,1,0,2,0.333333,0,1.000000
76902,2020-06-10,KT,50054,B,0,2,0,0,1,0,0,0.500000,0,0.500000


## 데이터프레임 변형: X값 그대로 Y값은 다음 경기의 값

In [5]:
df=pd.DataFrame()
for i in batter.P_ID.unique():
    temp = batter[batter['P_ID']==i].drop('AVG',axis=1)
    temp = temp.reset_index(drop='index')
    future_y = temp[['AB','HIT']]
    x = temp.drop(['AB','HIT'],axis=1)
    x = x.shift(periods=1,axis=0)
    new = pd.concat([x,future_y],axis=1)
    new = new.drop(0)
    df = pd.concat([df,new])

In [6]:
df = df.reset_index(drop='index')

In [7]:
df[df['P_ID']==50054].head()

,GDAY_DS,T_ID,P_ID,TB_SC,PA-AB,RUN,RBI,SH+SF,KK,SB_trial,BABIP,AB,HIT
0,2020-06-03,KT,50054.0,B,0.0,0.0,0.0,0.0,0.0,0.0,0.666667,3,0
1,2020-06-04,KT,50054.0,B,1.0,1.0,0.0,0.0,1.0,0.0,0.000000,0,0
2,2020-06-06,KT,50054.0,T,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,3,1
3,2020-06-07,KT,50054.0,T,0.0,0.0,0.0,0.0,2.0,0.0,1.000000,2,1
4,2020-06-10,KT,50054.0,B,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,4,3


## TEST SET 구성: 각 선수별 마지막 경기 X값

In [8]:
final_x = pd.DataFrame()
for i in batter.P_ID.unique():
    temp = batter[batter['P_ID']==i].drop(['AVG','AB','HIT'],axis=1).tail(1)
    final_x = pd.concat([final_x,temp])

In [9]:
final_x = final_x[final_x['GDAY_DS'].dt.year>=2020]

In [10]:
df = pd.concat([pd.get_dummies(df['TB_SC']),df],axis=1)
final_x = pd.concat([pd.get_dummies(final_x['TB_SC']),final_x],axis=1)

In [11]:
final_x[final_x['P_ID']==50054]

,B,T,GDAY_DS,T_ID,P_ID,TB_SC,PA-AB,RUN,RBI,SH+SF,KK,SB_trial,BABIP
81003,0,1,2020-07-19,KT,50054,T,0,0,0,0,1,0,0.0


In [12]:
test_x = final_x.drop(['GDAY_DS','T_ID','P_ID','TB_SC'],axis=1)

## 모델링

In [13]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')

In [14]:
'''
estimator = RandomForestRegressor(random_state=2020)

para_grids = {
            "n_estimators" : [100, 500, 1000],
            "max_depth" : [None, 10, 30, 50],
            "min_samples_split" : [2, 3, 5, 10],
            "min_samples_leaf" : [1, 3, 5, 10, 15],
            "max_features" : ["auto", "log2", "sqrt"],
            "bootstrap"    : [True, False]
        }

grid = GridSearchCV(estimator, para_grids, cv=3, n_jobs=-1, verbose=3, scoring = 'neg_mean_squared_error')
grid.fit(X_train, y_train)
forest = grid.best_estimator_

reg_prediction=forest.predict(X_val)

print (grid.best_score_, grid.best_params_)
'''


'\nestimator = RandomForestRegressor(random_state=2020)\n\npara_grids = {\n            "n_estimators" : [100, 500, 1000],\n            "max_depth" : [None, 10, 30, 50],\n            "min_samples_split" : [2, 3, 5, 10],\n            "min_samples_leaf" : [1, 3, 5, 10, 15],\n            "max_features" : ["auto", "log2", "sqrt"],\n            "bootstrap"    : [True, False]\n        }\n\ngrid = GridSearchCV(estimator, para_grids, cv=3, n_jobs=-1, verbose=3, scoring = \'neg_mean_squared_error\')\ngrid.fit(X_train, y_train)\nforest = grid.best_estimator_\n\nreg_prediction=forest.predict(X_val)\n\nprint (grid.best_score_, grid.best_params_)\n'

'bootstrap': True, 'max_depth': 10, 'max_features': 'auto', 'min_samples_leaf': 15, 'min_samples_split': 2, 'n_estimators': 1000}

## AB예측

In [15]:
X= df.drop(columns=['GDAY_DS','T_ID','P_ID','HIT','AB','TB_SC','HIT'])
y= df['AB']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, shuffle = True, random_state = 2020)

In [16]:
rf1 = RandomForestRegressor(bootstrap= True, max_depth= 10, max_features= 'auto', min_samples_leaf= 15, min_samples_split= 2, n_estimators=1000)
rf1.fit(X_train, y_train)

RandomForestRegressor(max_depth=10, min_samples_leaf=15, n_estimators=1000)

In [17]:
AB_predict = rf1.predict(X_val)

In [18]:
mse = mean_squared_error(y_val, AB_predict)
mse

2.04161351329212

In [19]:
AB = rf1.predict(test_x)

## HIT 예측

In [20]:
X= df.drop(columns=['GDAY_DS','T_ID','P_ID','HIT','AB','TB_SC','HIT'])
y= df['HIT']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, shuffle = True, random_state = 2020)

In [21]:
rf2 = RandomForestRegressor(bootstrap= True, max_depth= 10, max_features= 'auto', min_samples_leaf= 15, min_samples_split= 2, n_estimators=1000)
rf2.fit(X_train, y_train)

RandomForestRegressor(max_depth=10, min_samples_leaf=15, n_estimators=1000)

In [22]:
HIT_predict = rf2.predict(X_val)

In [23]:
mse = mean_squared_error(y_val, HIT_predict)
mse

0.7933674052241004

In [24]:
HIT = rf2.predict(test_x)

In [25]:
final_x['AB']=AB

In [26]:
final_x['HIT']=HIT

## AVG 예측

In [27]:
hit_by_team = final_x['HIT'].groupby(final_x['T_ID']).sum()
hit_by_team

T_ID
HH    22.997678
HT    17.962221
KT    16.055499
LG    21.217588
LT    17.750303
NC    20.092917
OB    17.546148
SK    18.241632
SS    19.207391
WO    21.142453
Name: HIT, dtype: float64

In [28]:
AB_by_team = final_x['AB'].groupby(final_x['T_ID']).sum()
AB_by_team

T_ID
HH    84.878777
HT    65.120728
KT    59.283320
LG    76.902329
LT    64.961731
NC    71.735365
OB    64.679808
SK    67.848471
SS    70.049176
WO    76.604002
Name: AB, dtype: float64

In [29]:
AVG_by_team = hit_by_team / AB_by_team
AVG_by_team

T_ID
HH    0.270947
HT    0.275830
KT    0.270827
LG    0.275903
LT    0.273242
NC    0.280098
OB    0.271277
SK    0.268858
SS    0.274199
WO    0.275997
dtype: float64

## 참고: 실제 데이터로 구한 AVG

In [30]:
batter = batter[batter['GDAY_DS'].dt.year>=2020]

In [31]:
hit_2020 = batter['HIT'].groupby(batter['T_ID']).sum()
ab_2020 = batter['AB'].groupby(batter['T_ID']).sum()

In [32]:
avg_2020 = hit_2020/ab_2020
avg_2020

T_ID
HH    0.239796
HT    0.275632
KT    0.290933
LG    0.279059
LT    0.274238
NC    0.290121
OB    0.302407
SK    0.242661
SS    0.274465
WO    0.270916
dtype: float64